## Installs, Imports and Configuration

In [1]:
from langchain.llms import OpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

In [2]:
# set OpenAI api key
API_KEY = "sk-..."

# setup language model
llm = OpenAI(openai_api_key=API_KEY)

## QA Pipeline
See https://python.langchain.com/docs/use_cases/question_answering/

### Loading Documents

In [3]:
pages = ["https://en.wikipedia.org/wiki/Vincent_van_Gogh",
         "https://medium.com/@anastasia_mze/vincent-van-gogh-from-a-bum-to-a-master-743aec22755e",
         "https://www.history.com/news/7-things-you-may-not-know-about-vincent-van-gogh",
         "https://arthinkal.com/vincent-van-gogh-a-brief-biography-1853-1890/",
         "https://www.metmuseum.org/toah/hd/gogh/hd_gogh.htm"]

loader = WebBaseLoader(pages)
docs = loader.load()

### Split

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
splits = text_splitter.split_documents(docs)

### Embed in Vector Store

In [5]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(openai_api_key=API_KEY))

### Query the DB

In [6]:
question = "When and where was Vincent van Gogh born?"
qa_docs = vectorstore.similarity_search(question)

In [7]:
for i in range(len(qa_docs)):
    print("-"*40)
    print(qa_docs[i].metadata["title"])
    print("-"*40)
    print(qa_docs[i].page_content)
    print("\n")

----------------------------------------
Vincent van Gogh - A Brief Biography (1853-1890) - Arthinkal Magazine
----------------------------------------
Vincent van Gogh was born on 30th March 1853 in Groot-Zundert, a municipality and town in the south of the Netherlands.
Van Gogh was the oldest surviving child of Theodorus van Gogh, a minister of the Dutch Reformed Church, and Anna Cornelia Carbentus, who came from a prosperous family.
Van Gogh was said to be a serious and thoughtful child, who was at first home-schooled by his mother and a governess. Later on, in 1860, he was sent to the local village school.


----------------------------------------
Vincent van Gogh - Wikipedia
----------------------------------------
Vincent Willem van Gogh was born on 30 March 1853 in Groot-Zundert, in the predominantly Catholic province of North Brabant in the Netherlands.[23] He was the oldest surviving child of Theodorus van Gogh (1822–1885), a minister of the Dutch Reformed Church, and his wif

### Generate Response

In [8]:
llm = ChatOpenAI(openai_api_key=API_KEY, model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectorstore.as_retriever())
qa = qa_chain({"query": question})

In [9]:
display(qa)

{'query': 'When and where was Vincent van Gogh born?',
 'result': 'Vincent van Gogh was born on 30th March 1853 in Groot-Zundert, a municipality and town in the south of the Netherlands.'}